<a href="https://colab.research.google.com/github/Venkata1236/Invoice-Processor/blob/main/2nd_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gspread oauth2client

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
print("Trying to connect...")

try:
    # This tells the program what it's allowed to access
    scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']

    # This uses your uploaded file to log in
    creds = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
    client = gspread.authorize(creds)

    # This tries to open your specific Google Sheet by its exact name
    sheet = client.open("Invoice Database")

    # If all the above steps worked, you will see this success message:
    print("\nSUCCESS! 🎉")
    print("Your code is successfully connected to your Google Sheet.")
    print("You are ready for the next part of the project.")

except Exception as e:
    # If any step failed, you will see this error message:
    print("\nERROR! Could not connect.")
    print("Here is the specific error message:", e)
    print("\n--- COMMON FIXES ---")
    print("1. Did you upload the 'credentials.json' file to this Colab session?")
    print("2. Did you share the Google Sheet with the correct 'client_email' address?")
    print("3. In the Google Sheet sharing settings, is the role for that email set to 'Editor'?")

Trying to connect...

SUCCESS! 🎉
Your code is successfully connected to your Google Sheet.
You are ready for the next part of the project.


In [ ]:
# Install tools for reading images and data validation
!sudo apt install tesseract-ocr
!pip install pytesseract
!pip install Pillow
!pip install pydantic

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
# === IMPORT LIBRARIES ===
import pytesseract
from PIL import Image
import re
from datetime import datetime
from pydantic import BaseModel, Field, validator
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# === DEFINE DATA STRUCTURE ===
class InvoiceData(BaseModel):
    supplier_name: str
    invoice_date: datetime
    total_amount: float = Field(gt=0)

    @validator('invoice_date', pre=True)
    def parse_date(cls, value):
        for fmt in ('%d-%m-%Y', '%B %d, %Y', '%d/%m/%Y'):
            try: return datetime.strptime(value, fmt)
            except ValueError: pass
        raise ValueError(f"Unable to parse date: {value}")

# === DEFINE FUNCTIONS ===
def extract_text_from_image(image_path):
    return pytesseract.image_to_string(Image.open(image_path))

# THIS FUNCTION IS NOW UPDATED FOR YOUR 'SIMPLEX SOLUTIONS' INVOICE
def parse_invoice_text(text: str):
    try:
        # For your invoice, the supplier is "SIMPLEX SOLUTIONS"
        supplier_name = "SIMPLEX SOLUTIONS"

        # The date is after the label "Invoice Date:"
        date_match = re.search(r'Invoice Date:\s*(\d{2}-\d{2}-\d{4})', text)
        date_str = date_match.group(1) if date_match else None

        # The total amount is after the label "TOTAL PRICE:"
        amount_match = re.search(r'TOTAL PRICE:\s*([\d,]+\.\d{2})', text, re.IGNORECASE)
        amount_str = amount_match.group(1).replace(',', '') if amount_match else None

        # Create the structured data
        return InvoiceData(supplier_name=supplier_name, invoice_date=date_str, total_amount=float(amount_str))
    except Exception as e:
        print(f"ERROR: Could not parse the invoice text. Details: {e}")
        return None

def write_to_google_sheet(data: InvoiceData):
    try:
        scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
        creds = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
        client = gspread.authorize(creds)
        worksheet = client.open("Invoice Database").worksheet("Sheet1")

        row = [
            data.supplier_name,
            data.invoice_date.strftime('%Y-%m-%d'),
            data.total_amount,
            "Pending Review",
            datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        ]
        worksheet.append_row(row)
        print("--> 🎉 Success! Data written to your Google Sheet.")
    except Exception as e:
        print(f"ERROR: Could not write to Google Sheet. Details: {e}")

# === RUN THE PROCESS ===
print("--- Starting Invoice Processor ---")
# Make sure this filename matches the file you uploaded
invoice_file = 'my_invoice.png'
raw_text = extract_text_from_image(invoice_file)

if raw_text:
    invoice_data = parse_invoice_text(raw_text)
    if invoice_data:
        write_to_google_sheet(invoice_data)
print("--- Processor Finished ---")

/tmp/ipython-input-16-3456599350.py:16: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  @validator('invoice_date', pre=True)


--- Starting Invoice Processor ---
--> 🎉 Success! Data written to your Google Sheet.
--- Processor Finished ---


In [ ]:
!pip install pdf2image
!sudo apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.8).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
# === 1. IMPORT LIBRARIES ===
import pytesseract
import re
import gspread
from PIL import Image
from datetime import datetime
from pdf2image import convert_from_path
from oauth2client.service_account import ServiceAccountCredentials
from pydantic import BaseModel, Field, field_validator # <-- Updated import for Pydantic v2

# === 2. CONFIGURATION ===
APPROVAL_THRESHOLD = 10000.00 # Invoices over this amount require manual review

# === 3. DEFINE DATA STRUCTURE (PYDANTIC V2 SYNTAX) ===
class InvoiceData(BaseModel):
    """A Pydantic model to structure and validate extracted invoice data."""
    supplier_name: str
    invoice_date: datetime
    total_amount: float = Field(gt=0) # Amount must be greater than 0

    @field_validator('invoice_date', mode='before') # <-- CORRECT Pydantic v2 syntax
    @classmethod
    def parse_date(cls, value):
        """Tries to parse various common date formats."""
        for fmt in ('%Y-%m-%d', '%d-%m-%Y', '%d/%m/%Y', '%b %d, %Y'):
            try:
                return datetime.strptime(str(value), fmt)
            except ValueError:
                pass
        raise ValueError(f"Unable to parse date: {value} with available formats")

# === 4. DEFINE CORE FUNCTIONS ===

def parse_invoice_text(text: str) -> InvoiceData | None:
    """
    Parses a single block of OCR text to extract structured invoice data.
    """
    try:
        # 1. Find Supplier Name
        supplier_match = re.search(r'Sold By:\s*(.*)', text, re.IGNORECASE)
        supplier_name = supplier_match.group(1).strip().replace(',', '') if supplier_match else None

        # 2. Find Invoice Date
        date_match = re.search(r'Invoice Date:\s*([\d-]+)', text, re.IGNORECASE)
        date_str = date_match.group(1).strip() if date_match else None

        # 3. Find Total Amount (IMPROVED REGEX)
        # re.DOTALL makes '.' match newlines, fixing the "Grand Total" amount being on the next line.
        amount_match = re.search(r'Grand Total.*?([\d,]+\.?\d*)', text, re.IGNORECASE | re.DOTALL)
        amount_str = amount_match.group(1).replace(',', '') if amount_match else None

        # --- VALIDATION AND ERROR HANDLING ---
        if not all([supplier_name, date_str, amount_str]):
            print("--- PARSING ERROR ---")
            if not supplier_name: print("Could not find Supplier Name (looked for 'Sold By:').")
            if not date_str: print("Could not find Invoice Date (looked for 'Invoice Date:').")
            if not amount_str: print("Could not find Total Amount (looked for 'Grand Total').")
            print("-----------------------")
            return None

        invoice_details = InvoiceData(
            supplier_name=supplier_name,
            invoice_date=date_str,
            total_amount=float(amount_str)
        )
        return invoice_details

    except Exception as e:
        print(f"An unexpected error occurred during parsing: {e}")
        return None

def process_invoice_approval(data: InvoiceData) -> str:
    """Determines the approval status based on the total amount."""
    if data.total_amount < APPROVAL_THRESHOLD:
        return "Auto-Approved"
    else:
        return "Pending Review"

def write_to_google_sheet(data: InvoiceData, approval_status: str):
    """Writes the final, validated data to Google Sheets."""
    try:
        scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
        creds = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
        client = gspread.authorize(creds)
        worksheet = client.open("Invoice Database").worksheet("Sheet1")

        row = [
            data.supplier_name,
            data.invoice_date.strftime('%Y-%m-%d'),
            data.total_amount,
            approval_status, # Use the determined approval status
            datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        ]
        worksheet.append_row(row)
        print(f"--> 🎉 Success! Data for '{data.supplier_name}' written to Google Sheet.")
    except Exception as e:
        print(f"ERROR: Could not write to Google Sheet. Details: {e}")

# === 5. RUN THE MAIN PROCESS ===
print("--- Starting Invoice Processor ---")
pdf_file = 'venkat_invoice.pdf'

try:
    print(f"1. Converting PDF '{pdf_file}' to images...")
    images = convert_from_path(pdf_file)

    # Combine text from all pages into a single string
    print("2. Reading text from all pages using OCR...")
    all_pages_text = [pytesseract.image_to_string(image) for image in images]
    full_text = "\n".join(all_pages_text) # <-- This joins all pages into one text block

    # Process the combined text
    if full_text:
        print("3. Parsing extracted text...")
        invoice_data = parse_invoice_text(full_text) # <-- CORRECT function name and variable

        if invoice_data:
            print("4. Processing approval status...")
            status = process_invoice_approval(invoice_data)
            print(f"   - Status for '{invoice_data.supplier_name}': {status}")

            print("5. Writing to Google Sheets...")
            write_to_google_sheet(invoice_data, status)
        else:
            print("   - Parsing failed. Skipping write to Google Sheets.")

except FileNotFoundError:
    print(f"\nERROR: The file '{pdf_file}' was not found. Please upload it to your Colab session.")
except Exception as e:
    print(f"An unexpected error occurred during PDF processing: {e}")

print("\n--- Processor Finished ---")

--- Starting Invoice Processor ---
1. Converting PDF 'venkat_invoice.pdf' to images...
2. Reading text from all pages using OCR...
3. Parsing extracted text...
4. Processing approval status...
   - Status for 'Tech-Connect Retail Private Limited ': Auto-Approved
5. Writing to Google Sheets...
--> 🎉 Success! Data for 'Tech-Connect Retail Private Limited ' written to Google Sheet.

--- Processor Finished ---


In [ ]:
# Create directories for our automated workflow
!mkdir -p invoices_to_process
!mkdir -p processed_invoices
print("Folders 'invoices_to_process' and 'processed_invoices' are ready.")

Folders 'invoices_to_process' and 'processed_invoices' are ready.


In [ ]:
# === 5. RUN THE MAIN PROCESS (AUTOMATED VERSION) ===
import os
import shutil # A library for file operations, like moving files

# Define the folders we will use
INPUT_FOLDER = 'invoices_to_process'
PROCESSED_FOLDER = 'processed_invoices'

print(f"--- Starting Automated Invoice Processor ---")
print(f"Looking for new invoices in folder: '{INPUT_FOLDER}'...")

# Get a list of all files in the input folder
try:
    invoice_files = os.listdir(INPUT_FOLDER)
except FileNotFoundError:
    print(f"ERROR: Input folder '{INPUT_FOLDER}' not found. Please create it first.")
    invoice_files = []

if not invoice_files:
    print("No new invoices to process.")
else:
    # Loop through each file found in the folder
    for filename in invoice_files:
        # Process only if it's a PDF file
        if filename.lower().endswith('.pdf'):
            pdf_path = os.path.join(INPUT_FOLDER, filename)
            print(f"\nProcessing file: {filename}")

            try:
                # 1. Convert PDF to images and extract text
                images = convert_from_path(pdf_path)
                full_text = "\n".join([pytesseract.image_to_string(img) for img in images])

                # 2. Parse the extracted text
                if full_text:
                    invoice_data = parse_invoice_text(full_text)

                    # 3. If parsing is successful, write to sheet and move the file
                    if invoice_data:
                        status = process_invoice_approval(invoice_data)
                        write_to_google_sheet(invoice_data, status)

                        # 4. Move the processed file to the 'processed' folder
                        shutil.move(pdf_path, os.path.join(PROCESSED_FOLDER, filename))
                        print(f"   -> Successfully processed and moved '{filename}'.")
                    else:
                        print(f"   -> Parsing failed for '{filename}'. File will not be moved.")
                else:
                    print(f"   -> OCR could not extract any text from '{filename}'.")

            except Exception as e:
                print(f"   -> An unexpected error occurred while processing '{filename}': {e}")
                # For robust error handling, you could move failed files to an 'error' folder here

print("\n--- Processor Finished ---")

--- Starting Automated Invoice Processor ---
Looking for new invoices in folder: 'invoices_to_process'...

Processing file: 3_invoice.pdf
--- PARSING ERROR ---
Could not find Supplier Name (looked for 'Sold By:').
Could not find Total Amount (looked for 'Grand Total').
-----------------------
   -> Parsing failed for '3_invoice.pdf'. File will not be moved.

--- Processor Finished ---
